In [8]:
import warnings
warnings.filterwarnings(action='ignore')

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num, WeekdayLocator, DayLocator, MONDAY
from mplfinance.original_flavor import candlestick_ohlc
import plotly.graph_objects as go
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps
from bokeh.plotting import output_notebook, show

from IPython.display import display
from dataclasses import dataclass, asdict
from typing import List, Annotated

from research_utils import plot_commodity, read_csv_imc, convert_to_candle, plot_commodities, make_continuos_df

output_notebook()
backtesting.set_bokeh_output(notebook=True)

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df_prices_day_0 = read_csv_imc(r"..\data\prices_round_1_day_0.csv")
df_prices_day_1 = read_csv_imc(r"..\data\prices_round_1_day_-1.csv")
df_prices_day_2 = read_csv_imc(r"..\data\prices_round_1_day_-2.csv")

prices_dict = {
    "day0": df_prices_day_0,
    "day1": df_prices_day_1,
    "day2": df_prices_day_2,
}

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\prices_round_1_day_0.csv'

In [4]:
df_ts_day_0 = read_csv_imc(r"..\data\trades_round_1_day_0_nn.csv")
df_ts_day_0.insert(0, 'day', -1)
df_ts_day_1 = read_csv_imc(r"..\data\trades_round_1_day_-1_nn.csv")
df_ts_day_1.insert(0, 'day', -1)
df_ts_day_2 = read_csv_imc(r"..\data\trades_round_1_day_-2_nn.csv")
df_ts_day_2.insert(0, 'day', -1)

time_and_sales_dict = {
    "day0": df_ts_day_0,
    "day1": df_ts_day_1,
    "day2": df_ts_day_2,
}

volume_dict = {
    "day0": df_ts_day_0.groupby(['symbol', 'timestamp', 'price']).agg(total_volume=('quantity', 'sum')).reset_index(),
    "day1": df_ts_day_1.groupby(['symbol', 'timestamp', 'price']).agg(total_volume=('quantity', 'sum')).reset_index(),
    "day2": df_ts_day_2.groupby(['symbol', 'timestamp', 'price']).agg(total_volume=('quantity', 'sum')).reset_index(),
}

In [27]:
tf = 160
ema1 = 8
ema2 = 11
STARFRUIT_candles_i = { day: convert_to_candle(prices_df=prices_dict[day], volume_df=volume_dict[day], product='STARFRUIT', interval=tf, bid_or_ask=True) for day in ["day0", "day1", "day2"] }
plot_commodity(STARFRUIT_candles_i["day0"], ema1=ema1, ema2=ema2, title=f"Starfruit (Bid) Day 0, {tf} timeframe")
plot_commodity(STARFRUIT_candles_i["day1"], ema1=ema1, ema2=ema2, title=f"Starfruit (Bid) Day 1, {tf} timeframe")
plot_commodity(STARFRUIT_candles_i["day2"], ema1=ema1, ema2=ema2, title=f"Starfruit (Bid) Day 2, {tf} timeframe")
plot_commodities(STARFRUIT_candles_i, continuous=True, area=True, yaxis_range=[4925, 5100], title=f"Starfruit (Bid) Days 0,1,2 {tf} timeframe", ema1=ema1, ema2=ema2)

In [28]:
def EMA(values, n):
    return pd.Series(values).ewm(span=n, adjust=True).mean()


class EMACrossover(Strategy):
    n1 = 8
    n2 = 11

    def init(self):
        self.ema1 = self.I(EMA, self.data.Close, self.n1)
        self.ema2 = self.I(EMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()
            
    def print_values(self):
        return f"n1: {self.n1} - n2: {self.n2}"

In [29]:
df = { day: convert_to_candle(prices_df=prices_dict[day], volume_df=volume_dict[day], product='STARFRUIT', interval=160, bid_or_ask=True) for day in ["day0", "day1", "day2"] }
df_test = make_continuos_df(df.copy())
df_test.set_index("time", inplace=True)
df_test.sort_index(inplace=True)

df_test = df_test.iloc[:-1]
bt = Backtest(df_test, EMACrossover, cash=1_000_000)
stats = bt.run()
stats

Start                                     0.0
End                                 2998680.0
Duration                            2998680.0
Exposure Time [%]                   99.928602
Equity Final [$]                    1108078.0
Equity Peak [$]                     1113131.0
Return [%]                            10.8078
Buy & Hold Return [%]                0.019837
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.189358
Avg. Drawdown [%]                   -0.142904
Max. Drawdown Duration              1151100.0
Avg. Drawdown Duration           12367.288136
# Trades                                888.0
Win Rate [%]                        43.355856
Best Trade [%]                       0.990884
Worst Trade [%]                     -0.197394
Avg. Trade [%]                    

In [30]:
bt.plot()

GridPlot(id='p5824', ...)

In [10]:
stats['_trades']["PnL_per_trade"] = (stats['_trades']["ExitPrice"] - stats['_trades']["EntryPrice"])
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,PnL_per_trade
0,198,3,30,5036.0,5044,1584.0,0.001589,22500,225000,202500,8.0
1,-198,30,31,5044.0,5046,-396.0,-0.000397,225000,232500,7500,2.0
2,198,31,33,5046.0,5045,-198.0,-0.000198,232500,247500,15000,-1.0
3,-198,33,34,5045.0,5048,-594.0,-0.000595,247500,255000,7500,3.0
4,198,34,37,5048.0,5050,396.0,0.000396,255000,277500,22500,2.0
5,-198,37,38,5050.0,5050,-0.0,-0.000000,277500,285000,7500,0.0
6,198,38,42,5050.0,5043,-1386.0,-0.001386,285000,315000,30000,-7.0
7,-198,42,46,5043.0,5048,-990.0,-0.000991,315000,345000,30000,5.0
8,197,46,49,5048.0,5046,-394.0,-0.000396,345000,367500,22500,-2.0
9,-197,49,50,5046.0,5047,-197.0,-0.000198,367500,375000,7500,1.0


In [11]:
stats_optimize, heatmap = bt.optimize(
    n1=range(1, 15, 1),
    n2=range(2, 500, 100),
    maximize="Return [%]",
    constraint=lambda param: param.n1 < param.n2,
    return_heatmap=True,
)
display(stats_optimize)
display(plot_heatmaps(heatmap, open_browser=False, plot_width=4000))
stats_optimize['_trades']["PnL_per_trade_per_contract"] = (stats_optimize['_trades']["ExitPrice"] - stats_optimize['_trades']["EntryPrice"]) * 100
display(stats_optimize['_trades'])

Start                                     0.0
End                                 4989600.0
Duration                            4989600.0
Exposure Time [%]                    99.25187
Equity Final [$]                    1023924.0
Equity Peak [$]                     1026730.0
Return [%]                             2.3924
Buy & Hold Return [%]                0.238569
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.079933
Avg. Drawdown [%]                   -0.263845
Max. Drawdown Duration              3197100.0
Avg. Drawdown Duration               300600.0
# Trades                                  5.0
Win Rate [%]                            100.0
Best Trade [%]                       1.008304
Worst Trade [%]                      0.059371
Avg. Trade [%]                    

GridPlot(id='p2118', ...)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,PnL_per_trade_per_contract
0,198,3,128,5036.0,5053,3366.0,0.003376,22500,960000,937500,1700.0
1,-198,128,190,5053.0,5050,594.0,0.000594,960000,2419800,1459800,-300.0
2,198,190,241,5050.0,5058,1584.0,0.001584,2419800,2802300,382500,800.0
3,-198,241,344,5058.0,5007,10098.0,0.010083,2802300,4569600,1767300,-5100.0
4,202,344,400,5007.0,5048,8282.0,0.008189,4569600,4989600,420000,4100.0


In [12]:
final_df = pd.read_excel(r"C:\Users\chris\imc-prosperity-TradeMonkeys\research\ema_pair_optimization_results_heatmap_for_3d_plot.xlsx")

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=final_df['n1'],
    y=final_df['n2'],
    z=final_df['Time Frame'],
    mode='markers',
    marker=dict(
        size=5,
        color=final_df['Return [%]'],  # set color to an array/list of desired values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8,
        colorbar=dict(title='Return [%]')
    )
)])

# Update plot layout
fig.update_layout(
    title='3D Heatmap of Returns vs n1, n2, and Time Frame',
    scene=dict(
        xaxis_title='n1',
        yaxis_title='n2',
        zaxis_title='Time Frame'
    ),
    autosize=False,
    width=1200,
    height=900,
    margin=dict(l=65, r=50, b=65, t=90)
)

# Show the figure
fig.show()

In [14]:
sorted_df = final_df.sort_values(by=["Return [%]"], ascending=False)
sorted_df.head(25)

,n1,n2,Return [%],Time Frame
36653,8,11,10.5983,160
36624,7,11,9.8615,160
26789,3,21,9.7579,120
24446,6,21,9.7376,110
24387,4,11,9.6069,110
41368,3,11,9.4455,180
12239,4,31,9.2966,60
4949,4,31,9.2966,30
7379,4,31,9.2966,40
17099,4,31,9.2966,80
